In [1]:
import numpy as np
import pandas as pd
import os
!pip install datasets peft torch torchvision
!pip install -q transformers torch evaluate scikit-learn pillow
import ast
from PIL import Image
import torch
from sklearn.model_selection import train_test_split 
import evaluate                                        
from transformers import Blip2Processor, Blip2ForConditionalGeneration , BitsAndBytesConfig, get_linear_schedule_with_warmup
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

CSV_PATH = "/kaggle/input/vrmini2/12049.csv"  
df = pd.read_csv(CSV_PATH)

# Renamed for easy understanding
df = df.rename(columns={"full_path": "image_path"})

# Check image existence
exists = df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
df = df[exists].reset_index(drop=True)

# Final cleaned dataframe
df = df[["image_path", "question", "answer"]]
print(f"Total examples: {len(df)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing

2025-05-17 03:36:56.315548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747453016.519930      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747453016.575113      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Skipping 0 rows with missing images
Total examples: 12049


In [2]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model     = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float32,
    device_map="auto"
)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model.eval()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

trainable params: 473,088 || all params: 3,942,919,680 || trainable%: 0.0120


PeftModel(
  (base_model): LoraModel(
    (model): Blip2ForConditionalGeneration(
      (vision_model): Blip2VisionModel(
        (embeddings): Blip2VisionEmbeddings(
          (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
        )
        (encoder): Blip2Encoder(
          (layers): ModuleList(
            (0-38): 39 x Blip2EncoderLayer(
              (self_attn): Blip2Attention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear(in_features=1408, out_features=4224, bias=True)
                (projection): Linear(in_features=1408, out_features=1408, bias=True)
              )
              (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
              (mlp): Blip2MLP(
                (activation_fn): GELUActivation()
                (fc1): Linear(in_features=1408, out_features=6144, bias=True)
                (fc2): Linear(in_features=6144, out_features=1408, bias=True)
              )
              

In [5]:
from torch.utils.data import DataLoader
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df, processor, max_length=32):
        self.df = df
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row.image_path).convert("RGB")
        question, answer = row.question, row.answer
        
        enc = self.processor(
            image, question,
            padding="max_length", truncation=True,
            max_length=self.max_length, return_tensors="pt"
        )
        labels = self.processor.tokenizer(
            answer,
            padding="max_length", truncation=True,
            max_length=self.max_length, return_tensors="pt"
        ).input_ids
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        item = {
            "pixel_values": enc.pixel_values.squeeze(),
            "input_ids":    enc.input_ids.squeeze(),
            "attention_mask": enc.attention_mask.squeeze(),
            "labels":       labels.squeeze()
        }
        return item

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_ds = VQADataset(train_df, processor)
val_ds   = VQADataset(val_df, processor)

train_dataloader = DataLoader(train_ds, batch_size=12, shuffle=True,  pin_memory=True)
valid_dataloader   = DataLoader(val_ds,   batch_size=12, shuffle=False, pin_memory=True)



In [6]:
from tqdm import tqdm
import pickle

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5) 
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1, verbose=False)
num_epochs = 3
patience = 6
min_eval_loss = float("inf")
early_stopping_hook = 0
tracking_information = []



max_grad_norm = 1.0
output_dir = "/kaggle/working/blip2_with_lora"
os.makedirs(output_dir, exist_ok=True)

#Training Loop
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for idx, batch in zip(tqdm(range(len(train_dataloader)), desc=f'Epoch {epoch+1} Training'), train_dataloader):
        try:
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_mask = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)
        except KeyError as e:
            print(f"Missing key in batch: {e}")
            continue # Skip batch if missing keys
        except Exception as e:
            print(f"Error loading batch data: {e}")
            continue

        optimizer.zero_grad()

        
        outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_mask, 
                            labels=labels)
        loss = outputs.loss

        if loss is None:
             print(f"Warning: Loss is None for batch {idx}. Skipping backward pass.")
             continue

        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Warning: Detected NaN/Inf loss at batch {idx} before scaling. Skipping batch.")
            continue # Skip the backward pass for current batch

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        

        # Log loss
        if not (torch.isnan(loss) or torch.isinf(loss)):
             epoch_loss += loss.item()
        else:
             print(f"NaN/Inf loss recorded at step {idx}, not adding to epoch_loss")

    #Validation
    model.eval()
    eval_loss = 0
    with torch.no_grad(): # since validation
        for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc=f'Epoch {epoch+1} Validating'), valid_dataloader):
            try:
                input_ids = batch.pop('input_ids').to(device)
                pixel_values = batch.pop('pixel_values').to(device)
                attention_mask = batch.pop('attention_mask').to(device)
                labels = batch.pop('labels').to(device)
            except KeyError as e:
                print(f"Missing key in validation batch: {e}")
                continue
            except Exception as e:
                print(f"Error loading validation batch data: {e}")
                continue
            outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_mask,
                            labels=labels)
            loss = outputs.loss

            if loss is not None and not (torch.isnan(loss) or torch.isinf(loss)):
                eval_loss += loss.item()
            else:
                 print(f"NaN/Inf/None loss detected during validation step {idx}")

    # Calculate average losses and handle division by zero
    avg_train_loss = epoch_loss / len(train_dataloader) if len(train_dataloader) > 0 else float('nan')
    avg_eval_loss = eval_loss / len(valid_dataloader) if len(valid_dataloader) > 0 else float('nan')
    current_lr = optimizer.param_groups[0]["lr"]

    print(f"Epoch: {epoch+1} - Training loss: {avg_train_loss:.4f} - Eval Loss: {avg_eval_loss:.4f} - LR: {current_lr}")

    # Check for NaN/Inf 
    if not (torch.isnan(torch.tensor(avg_eval_loss)) or torch.isinf(torch.tensor(avg_eval_loss))): # Conversion to tensor
        tracking_information.append((avg_train_loss, avg_eval_loss, current_lr))
        scheduler.step() 
        
        if avg_eval_loss < min_eval_loss:
            best_model_dir = os.path.join(output_dir, "best-checkpoint")
            # save_pretrained on a PeftModel saves the adapter config and weights
            model.save_pretrained(best_model_dir)
            print(f"Saved PEFT adapter checkpoint to {best_model_dir}")
            min_eval_loss = avg_eval_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            print(f"Validation loss did not improve. Early stopping counter: {early_stopping_hook}/{patience}")
            if early_stopping_hook >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break
    else:
        print(f"Skipping saving and scheduler step due to NaN/Inf validation loss in Epoch {epoch+1}.")

pickle.dump(tracking_information, open("tracking_information.pkl", "wb"))
print("Finetuning completed!!!")


Epoch 1 Validating: 100%|██████████| 201/201 [14:47<00:00,  4.41s/it]


Epoch: 1 - Training loss: 3.2802 - Eval Loss: 1.9551 - LR: 1e-05
Saved PEFT adapter checkpoint to /kaggle/working/blip2_with_lora/best-checkpoint


Epoch 2 Validating: 100%|██████████| 201/201 [14:47<00:00,  4.41s/it]


Epoch: 2 - Training loss: 1.9369 - Eval Loss: 1.5541 - LR: 9.5e-06
Saved PEFT adapter checkpoint to /kaggle/working/blip2_with_lora/best-checkpoint


Epoch 3 Validating: 100%|██████████| 201/201 [14:47<00:00,  4.42s/it]

Epoch: 3 - Training loss: 1.7098 - Eval Loss: 1.4250 - LR: 9.025e-06
Saved PEFT adapter checkpoint to /kaggle/working/blip2_with_lora/best-checkpoint
Finetuning completed!!!


In [7]:
from peft import PeftModel

output_dir = "/kaggle/working/blip2_with_lora"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
base_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float32,
    device_map="auto"
)
adapter_path = os.path.join(output_dir, "best-checkpoint")
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()
model_device = next(model.parameters()).device

preds, refs = [], []
for row in tqdm(val_df.itertuples(), total=len(val_df), desc="Generating Predictions"):
    try:
        img = Image.open(row.image_path).convert("RGB")
        prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
        inputs = processor(images=img, text=prompt, return_tensors="pt")
        inputs = {k: v.to(model_device) for k, v in inputs.items()}

        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=10, do_sample=False)
        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        pred_words = text.split()
        pred = pred_words[0].rstrip(".,;:!?") if pred_words else ""
        preds.append(pred.lower())
        refs.append(str(row.answer).lower())
    except FileNotFoundError:
        print(f"Warning: Image not found at {row.image_path}. Skipping.")
        preds.append("[ImageNotFound]")
        refs.append(str(row.answer).lower())
    except Exception as e:
        print(f"Error processing row {row.Index}: {e}")
        preds.append("[Error]")
        refs.append(str(row.answer).lower())
print("Done!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Predictions: 100%|██████████| 2410/2410 [16:55<00:00,  2.37it/s]

Done!


In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from sentence_transformers import SentenceTransformer, util 
import numpy as np
import pandas as pd 

# Normalization process
preds_l = [str(p).lower().strip() for p in preds] 
refs_l  = [str(r).lower().strip() for r in refs] 

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.458
Exact-match Precision: 1.000
Exact-match Recall:    0.458
Exact-match F1:        0.628



In [22]:
import torch
from tqdm import tqdm

# Set device explicitly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Ensure model is on GPU

preds, refs = [], []
model_dtype = next(model.parameters()).dtype
model_device = next(model.parameters()).device  # Should be 'cuda:0' if using GPU

for row in tqdm(train_df.itertuples(), total=len(train_df), desc="Generating Predictions"):
    try:
        img = Image.open(row.image_path).convert("RGB")
        prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
        inputs = processor(images=img, text=prompt, return_tensors="pt")
        # Move tensors to GPU and correct dtype
        inputs = {k: v.to(model_device, dtype=model_dtype) if v.dtype.is_floating_point else v.to(model_device) for k, v in inputs.items()}
        out = model.generate(**inputs, max_new_tokens=10)
        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        pred = text.split()[0].rstrip(".,;:!?") if text else ""
        preds.append(pred.lower())
        refs.append(str(row.answer).lower())
    except FileNotFoundError:
        print(f"Warning: Image not found at {row.image_path}. Skipping.")
        preds.append("[ImageNotFound]")
        refs.append(str(row.answer).lower())
    except Exception as e:
        print(f"Error processing row {row.Index}: {e}")
        preds.append("[Error]")
        refs.append(str(row.answer).lower())


Generating Predictions: 100%|██████████| 9639/9639 [36:42<00:00,  4.38it/s]


In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from sentence_transformers import SentenceTransformer, util 
import numpy as np
import pandas as pd 

# Normalization process
preds_l = [str(p).lower().strip() for p in preds] 
refs_l  = [str(r).lower().strip() for r in refs] 

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.351
Exact-match Precision: 1.000
Exact-match Recall:    0.351
Exact-match F1:        0.519



In [2]:
import torch
from tqdm import tqdm

# Set device explicitly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Ensure model is on GPU

preds, refs = [], []
model_dtype = next(model.parameters()).dtype
model_device = next(model.parameters()).device  # Should be 'cuda:0' if using GPU

for row in tqdm(df.itertuples(), total=len(df), desc="Generating Predictions"):
    try:
        img = Image.open(row.image_path).convert("RGB")
        prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
        inputs = processor(images=img, text=prompt, return_tensors="pt")
        # Move tensors to GPU and correct dtype
        inputs = {k: v.to(model_device, dtype=model_dtype) if v.dtype.is_floating_point else v.to(model_device) for k, v in inputs.items()}
        out = model.generate(**inputs, max_new_tokens=10)
        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        pred = text.split()[0].rstrip(".,;:!?") if text else ""
        preds.append(pred.lower())
        refs.append(str(row.answer).lower())
    except FileNotFoundError:
        print(f"Warning: Image not found at {row.image_path}. Skipping.")
        preds.append("[ImageNotFound]")
        refs.append(str(row.answer).lower())
    except Exception as e:
        print(f"Error processing row {row.Index}: {e}")
        preds.append("[Error]")
        refs.append(str(row.answer).lower())


NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from sentence_transformers import SentenceTransformer, util 
import numpy as np
import pandas as pd 

# Normalization process
preds_l = [str(p).lower().strip() for p in preds] 
refs_l  = [str(r).lower().strip() for r in refs] 

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

In [9]:
import shutil
shutil.make_archive('/kaggle/working/best-checkpoint', 'zip', '/kaggle/working/blip2_with_lora/best-checkpoint')


'/kaggle/working/best-checkpoint.zip'